# PyOccam Demo - Getting Started

This notebook walks through the standard OCCAM workflow:
1. Load data
2. Search for the best model
3. Examine search results
4. Fit the best model
5. Extract confusion matrix metrics

Uses the bundled dementia dataset (424 patients, 18 features).
For advanced features, see `pyoccam_demo_advanced.ipynb`

## Step 1: Load Data

In [6]:
import pyoccam
print(f"PyOccam {pyoccam.__version__}")

data = pyoccam.load_dementia()

print(f"\nDataset:  {data}")
print(f"Samples:  {data.n_samples}")
print(f"Features: {data.n_features}")
print(f"Target:   {data.target_name}")
print(f"\nFeature variables:")
for i, name in enumerate(data.feature_names):
    print(f"  {i+1:2d}. {name}")

PyOccam 0.9.5
[OK] Loaded dementia: 424 samples, 18 features

Dataset:  OccamData(n_samples=424, n_features=18, target='CaseControl')
Samples:  424
Features: 18
Target:   CaseControl

Feature variables:
   1. APOE
   2. Gender
   3. Education
   4. AgeLastExam
   5. rs1801133
   6. rs3818361
   7. rs7561528
   8. rs744373
   9. rs6943822
  10. rs4298437
  11. rs7012010
  12. rs11136000
  13. rs10786998
  14. rs11193130
  15. rs610932
  16. rs3851179
  17. rs3764650
  18. rs3865444


## Step 2: Search for Best Model

Run a loopless-up beam search through the model lattice.

In [7]:
manager = data.manager

print("Running loopless-up search (levels=5, width=3)...")
search_report = manager.generate_search_report(
    search_type="full-up",
    levels=5,
    width=3
)

Running loopless-up search (levels=5, width=3)...
Level 1/5: 18 new models, 3 kept (total: 4)
Level 2/5: 48 new models, 3 kept (total: 7)
Level 3/5: 48 new models, 3 kept (total: 10)
Level 4/5: 52 new models, 3 kept (total: 13)
Level 5/5: 57 new models, 3 kept (total: 16)


## Step 3: Examine Search Results

OCCAM can select the best model by three criteria:
- **BIC**: Most conservative, penalizes complexity heavily
- **AIC**: Moderate complexity penalty
- **Information**: Best information capture with alpha < 0.05

In [8]:
best_bic = manager.get_best_model_by_bic()
best_aic = manager.get_best_model_by_aic()
best_info = manager.get_best_model_by_information()

print(f"Best by BIC (conservative):  {best_bic}")
print(f"Best by AIC (moderate):      {best_aic}")
print(f"Best by Information:         {best_info}")

Best by BIC (conservative):  IV:ApZ:EdZ:CZ
Best by AIC (moderate):      IV:ApZ:EdZ:AgZ:CZ:KZ
Best by Information:         IV:ApZ:EdZ:AgZ:CZ:KZ


## Step 4: Fit the Best Model

Generate a detailed fit report including conditional probability tables
and confusion matrix.

In [9]:
# AIC strikes a nice balance -- more interesting than the conservative BIC
print(f"Fitting model: {best_aic}")
fit_report = manager.generate_fit_report(best_aic, target_state="0")

# Print the full fit report -- includes conditional DV tables
# and confusion matrix with all statistics
print(fit_report)

Fitting model: IV:ApZ:EdZ:AgZ:CZ:KZ
Sample size: 424
Variables: 19

    Model  IV:ApZ:EdZ:AgZ:CZ:KZ (Directed System)
    IV Component:  APOE; Gender; Education; AgeLastExam; rs1801133; rs3818361; rs7561528; rs744373; rs6943822; rs4298437; rs7012010; rs11136000; rs10786998; rs11193130; rs610932; rs3851179; rs3764650; rs3865444  ApSxEdAgABCDEFGHJKLMNP
    Model Component:   APOE; CaseControl  ApZ
    Model Component:   Education; CaseControl  EdZ
    Model Component:   AgeLastExam; CaseControl  AgZ
    Model Component:   rs7561528; CaseControl  CZ
    Model Component:   rs11193130; CaseControl  KZ
    Degrees of Freedom (DF):  -1
    Loops:  YES
    Entropy(H):  9.5576
    Information captured (%):  16.9239
    Transmission (T):  0.829681

-------------------------------------------------------------------------

    REFERENCE = TOP
      Value  Prob. (Alpha)
    Log-Likelihood (LR)  487.677  0
    Pearson X2  278.62  
    Delta DF (dDF)  4  

-------------------------------------------

## Step 5: Confusion Matrix Metrics

Extract classification performance metrics as a Python dictionary.

In [10]:
cm = manager.get_confusion_matrix(best_aic, target_state="0")

if cm.get('has_values', False):
    print(f"Model:       {best_aic}")
    print(f"Accuracy:    {cm['train_accuracy']:.1%}")
    print(f"Sensitivity: {cm['train_sensitivity']:.1%}")
    print(f"Specificity: {cm['train_specificity']:.1%}")
    print(f"Precision:   {cm['train_precision']:.1%}")
    print(f"F1 Score:    {cm['train_f1_score']:.1%}")
    print(f"\n  TP={cm['train_tp']:4.0f}  FP={cm['train_fp']:4.0f}")
    print(f"  FN={cm['train_fn']:4.0f}  TN={cm['train_tn']:4.0f}")

Model:       IV:ApZ:EdZ:AgZ:CZ:KZ
Accuracy:    71.7%
Sensitivity: 62.6%
Specificity: 80.1%
Precision:   74.3%
F1 Score:    67.9%

  TP= 127  FP=  44
  FN=  76  TN= 177


## Next Steps

```python
# Try the landslides dataset
data = pyoccam.load_landslides()

# Load your own OCCAM-format data
data = pyoccam.load_data("mydata.txt")

# Convert CSV to OCCAM format
output_file, data = pyoccam.make_occam_input_from_csv("mydata.csv")

# Quick reference
pyoccam.help()
```

See `pyoccam_demo_advanced.ipynb` for CSV conversion, model comparison,
and batch processing with the real landslides dataset.